Brief Description of the Assignment:

This Web Scrapper is extracting data from multiple sources, 'List_of_United_States_cities_by_population' being the primary Source page. Also, this Scrapper is extracting for all City Infobox geophcard vcard details.

There are total 315 (1 primary and 314 City links) sites from which data is being gathered, cleaned and stored into 7 separate CSV files. There is one big CSV files which contains all city details and all other 6 files provides insights on overall picture of Cities of US by population.

This Scrapper can be used to extract lot of of information besides its development scope with little modification.

ReadMe:

1) Please install all the packages that are pre-installed in designated system

2) CSV Files Path Shoulde be changed to as per the User speicification before running to_csv commands

In [ ]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from functools import reduce
import itertools
import re
import warnings
from collections import Counter

In [630]:
warnings.filterwarnings("ignore")

In [632]:
url = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
html = requests.get(url).content
df_list = pd.read_html(html)
df1 = df_list[-1]
df2 = df_list[-2]
df3 = df_list[-4]
df4 = df_list[-6]
df5 = df_list[-7]
df6 = df_list[-8]

In [633]:
list1 = np.array(df1.iloc[:1])
for i in range(len(list1[0])):
    df1 = df1.rename(index=int,columns = {i:list1[0][i]})
df1 = df1.iloc[1:]

In [634]:
df2 = df2.drop([6, 8], axis=1)
df2 = df2.rename(index = int,columns = {9:'ANSI',10:'Location'})
list2 = np.array(df2.iloc[:1])
for i in range(len(list2[0])):
    df2 = df2.rename(index=int,columns = {i:list2[0][i]})
df2 = df2.iloc[1:]
df2.columns = df2.columns.fillna('2010 popultaion density')

In [635]:
df3 = df3.drop([6,8],axis=1)
list3 = np.array(df3.iloc[:1])
df3 = df3.rename(index = int,columns = {9:'Location'})
for i in range(len(list3[0])):
    df3 = df3.rename(index=int,columns = {i:list3[0][i]})
df3 = df3.iloc[1:]
df3.columns = df3.columns.fillna('2017 popultaion density')

In [636]:
list4 = np.array(df4.iloc[:1])
for i in range(len(list4[0])):
    df4 = df4.rename(index=int,columns = {i:list4[0][i]})
df4 = df4.iloc[1:]

In [637]:
list5 = np.array(df5.iloc[:1])
for i in range(len(list5[0])):
    df5 = df5.rename(index=int,columns = {i:list5[0][i]})
df5 = df5.iloc[1:]

In [638]:
df6 = df6.drop([7,9],axis=1)
list6 = np.array(df6.iloc[:1])
df6 = df6.rename(index = int,columns = {10:'Location'})
for i in range(len(list6[0])):
    df6 = df6.rename(index=int,columns = {i:list6[0][i]})
df6 = df6.iloc[1:]
df6.columns = df6.columns.fillna('2016 popultaion density')

In [639]:
soup = BeautifulSoup((requests.get(url)).text,'html.parser')

In [640]:
row = soup.find_all('tr')

In [641]:
header = ([i.text for i in row[19].find_all("th")])
header = [i.strip() for i in header]

In [642]:
links = []
for i in range(20,334):
    links.append('https://en.wikipedia.org'+ row[i].find('a')['href'])


# City Data

In [652]:
def remove_all(the_list, val):
        while val in the_list:
            the_list.remove(val)

In [653]:
def addRow(df,ls):
    df.loc[len(df)] = ls
    return df

In [654]:
def City_Scrapper(city_soup): 
    
    t= 100
    try:
        for i in range(len(city_soup.find_all('table'))): 
            if city_soup.find_all('table')[i]['class'] == ['infobox', 'geography', 'vcard']:
                t = i
                break
    except: 
        for i in range(i+1,len(city_soup.find_all('table'))): 
            if city_soup.find_all('table')[i]['class'] == ['infobox', 'geography', 'vcard']:
                t = i
                break
    #### Finding the starting point for scrapper to start ####
    for j in range(len(city_soup.find_all('table')[t].find_all('tr'))):
        if city_soup.find_all('table')[t].find_all('tr')[j].contents[0].text == 'Country':
            start = j
            break
    for j in range(len(city_soup.find_all('table')[t].find_all('tr'))):
        if city_soup.find_all('table')[t].find_all('tr')[j].contents[0].text == 'Website':
            end = j+1
            break
    table = city_soup.find_all('table')[t].find_all('tr')[start:end]
    
    #### Extracting the columns for DataFrame ####
    col= []
    value = []
    try:
        for i in range(0,len((table))):
            if len(table[i].contents) == 1:
                value = table[i].th.contents[0]
                col.append(['SPLITTER'])
                continue
            elif '•' in table[i].contents[0].text: 
                col.append([(value+table[i].contents[0].text).replace('•',' ')])
            elif len(table[i].contents) > 1 and '•' not in table[i].contents[0]:
                col.append([table[i].contents[0].text])
    except Exception as e:
        print(logger.exception(e))
    col = reduce(lambda x,y:x+y,col)
    col = [w.replace('\xa0', ' ') for w in col]
    col = [re.sub('((\[)[0-9]*\w*(\]))','',w) for w in col]
    col = [w.replace('\n', ' ') for w in col]
    remove_all(col, 'SPLITTER')
    
     #### Extracting the Data for DataFrame ####
    row= []
    try:
        for i in range(0,len((table))):
            if len(table[i].contents) == 1:
                continue
            else:
                row.append([table[i].contents[1].text])
    except Exception as e:
        pass
    row = reduce(lambda x,y:x+y,row)
    row = [w.replace('\xa0', ' ') for w in row]
    row = [re.sub('((\[)[0-9]*\w*(\]))','',w) for w in row]
    row = [w.replace('\n', ' ') for w in row]
    return(col,row)
    

In [655]:
## Cleaning Columns
def cleaning(uncleaned_col):
    uncleaned_col = [w.replace('(s)', ' ') for w in uncleaned_col]
    uncleaned_col = [w.upper() for w in uncleaned_col]
    uncleaned_col = [w.strip() for w in uncleaned_col]  
    return(uncleaned_col)

In [691]:
# Storing all parsed data into one list
df = []
for city_link in links:
    city_soup = BeautifulSoup((requests.get(city_link)).text,'html.parser')
    
    mylist = cleaning(City_Scrapper(city_soup)[0])
    counts = Counter(mylist) 
    for s,num in counts.items():
        if num > 1: # ignore strings that only appear once
            for suffix in range(1, num + 1): # suffix starts at 1 and increases by 1 each time
                mylist[mylist.index(s)] = s + str(suffix) # replace each appearance of s
                
    City_df = pd.DataFrame()
    for i in range(len(mylist)):
        City_df[mylist[i]] = mylist[i]
    addRow(City_df,City_Scrapper(city_soup)[1])
    df.append(City_df)

In [715]:
## Adding City to DataFrame
df6['City'] =[re.sub('((\[)\w*])','',w) for w in df6['City']]
for i in range(len(df)):
    df[i]['City'] = df6['City'][i+1]

In [716]:
## Getting final Column for csv
columns = []
clean_col= []
uni_col = []
for i in range(len(df)-1):
    columns.append((list(df[i].columns)+list(df[i+1].columns)))
uni_col = set(sum(columns,[]))
uni_col = list(uni_col)
for i in range(1,len(uni_col)):
    if uni_col[i][-1] == 'S' and uni_col[i][0:-1] in uni_col:
        clean_col.append([uni_col[i][0:-1]])
    else:
        clean_col.append([uni_col[i]])
clean_col = set(sum(clean_col,[]))
clean_col = list(clean_col)
clean_col = [re.sub('((\[)+)','',w) for w in clean_col]

In [717]:
## Dropping unecessary columns
for i in range(len(df)):
    for l in df[i].columns:
        if l== '':
            df[i].drop('', axis=1, inplace=True)

In [718]:
## Cleaning Columns in DataFrame
for i in range(len(df)):
    for l in df[i].columns:
        l = re.sub('([(\[))(\]])','',l)
        df[i]=df[i].rename(columns={l: l})
        if l[-1] == 'S' and l[0:-1] in sum(columns,[]):
            df[i]=df[i].rename(columns={l: l[0:-1]})


In [719]:
## Cleaning Columns in other DataFrames
df1 = df1.rename(columns={w:re.sub('((\[)\w*])','',w) for w in df1.columns})
df2 = df2.rename(columns={w:re.sub('((\[)\w*])','',w) for w in df2.columns})
df3 = df3.rename(columns={w:re.sub('((\[)\w*])','',w) for w in df3.columns})
df4 = df4.rename(columns={w:re.sub('((\[)\w*])','',w) for w in df4.columns})
df5 = df5.rename(columns={w:re.sub('((\[)\w*])','',w) for w in df5.columns})
df6 = df6.rename(columns={w:re.sub('((\[)\w*])','',w) for w in df6.columns})
df1['2018 estimated population'] =[re.sub('((\[)\w*])','',w) for w in df1['2018 estimated population']]

In [789]:
# Creating final DataFrame
final_df = pd.DataFrame()
for i in range(len(clean_col)):
    final_df[clean_col[i]] = clean_col[i]

In [790]:
# Storing Data in final DataFrame
for i in range(len(df)):
    final_df = final_df.append(df[i], ignore_index=True)
final_df = final_df.dropna(how='all', axis=1)
final_df = final_df.rename(columns={w:re.sub('([(\[))(\]])','',w) for w in final_df.columns})
final_df = final_df.replace(np.nan, '', regex=True)
final_df = final_df.drop(['CITY','APPROVED','A CITY'],axis=1)
final_df['AREA CONSOLIDATED CITY-COUNTY'] = ["{}{}".format(b_, a_) for a_, b_ in zip(list(final_df['AREA   CONSOLIDATED CITY-PARISH']), list(final_df['AREA   CONSOLIDATED CITY-PARISHN 1']))]
final_df['AREA CONSOLIDATED CITY-COUNTY'] = ["{}{}".format(b_, a_) for a_, b_ in zip(list(final_df['AREA   CONSOLIDATED CITY-COUNTY']), list(final_df['AREA   CONSOLIDATED CITY–COUNTY']))]
final_df['AREA TOTAL'] = ["{}{}".format(b_, a_) for a_, b_ in zip(list(final_df['AREA   TOTAL']), list(final_df['AREA   TOTALN 2']))]
final_df = final_df.drop(['AREA   CONSOLIDATED CITY-PARISH','AREA   CONSOLIDATED CITY-PARISHN 1',\
                          'AREA   CONSOLIDATED CITY-COUNTY','AREA   CONSOLIDATED CITY–COUNTY',\
                         'AREA   TOTAL','AREA   TOTALN 2'],axis=1)

# Exporting data to CSV

In [744]:
df1.to_csv('A:\Crawler df/Cityover10k.csv')
df2.to_csv("A:\Crawler df/CDP's.csv")
df3.to_csv("A:\Crawler df/Puerto Rico's municipalities.csv")
df4.to_csv("A:\Crawler df/State and City Nos.csv")
df5.to_csv("A:\Crawler df/Pop vs Number of municipal governments.csv")
df6.to_csv("A:\Crawler df/City Details.csv")
final_df.to_csv('A:\Crawler df/City Infobox.csv')